In [ ]:
from pyDigitalWaveTools.vcd.parser import VcdParser
import pandas as pd

def load_signals_to_df(names, data):
    max_time = max([max([kv[0] for kv in d]) for d in data])

    df = None
    for i in range(len(data)):
        name = names[i]
        d = data[i]

        tmp = pd.DataFrame(d, columns=['time', name]).set_index(['time'])
        tmp = tmp.reindex(pd.RangeIndex(0, max_time+1), method='ffill')

        if i == 0:
            df = tmp
        else:
            df = df.join(tmp)
            
    return df

def load_trace(filename, signals):
    with open(filename) as fp:
        vcd = VcdParser()
        vcd.parse(fp)

        names = [signal[-1] for signal in signals]
        data = []
        for signal in signals:
            trace = vcd.scope.children[signal[0]]
            for path in signal[1:]:
                trace = trace.children[path]
            data.append(trace.data)

        return load_signals_to_df(names, data)

## Trace generation

1. Run the following command in the root directory to generate sim_shufflev.fst 

    ```
    ./build/lowrisc_ibex_demo_system_0/sim-verilator/Vibex_demo_system -t --meminit=ram,tfmicro_sine,elf && cp sim.fst sim_shufflev.fst && rm sim.fst
    ```

    or (for customize the random seed)

    ```
    ./build/lowrisc_ibex_demo_system_0/sim-verilator/Vibex_demo_system -t --meminit=ram,tfmicro_sine,elf +verilator+seed+2 && cp sim.fst sim_shufflev.fst && rm sim.fst
    ```

2. Open the trace in gtkwave and load the setting file (util/sim_shufflev.gtkw)

In [ ]:
ibex_trace_df = load_trace('../sim_ibex.vcd', [['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'clk_i'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'addr_o[31:0]'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'rdata_o[31:0]'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'ready_i'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'valid_o'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'branch_i'],
                                               ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'instr_decompressed[31:0]']])
shufflev_trace_df = load_trace('../sim_shufflev.vcd', [['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'clk_i'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'addr_o[31:0]'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'rdata_o[31:0]'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'ready_i'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'valid_o'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'branch_i'],
                                                       ['TOP', 'ibex_demo_system', 'u_top', 'u_ibex_core', 'if_stage_i', 'gen_prefetch_buffer', 'prefetch_buffer_i', 'fetch_id_idex[31:0]']])

ibex_trace_df_filter = ibex_trace_df[(ibex_trace_df['clk_i'] == '1') & (ibex_trace_df['ready_i'] == '1') & (ibex_trace_df['valid_o'] == '1') & (ibex_trace_df['branch_i'] == '0')]
shufflev_trace_df_filter = shufflev_trace_df[(shufflev_trace_df['clk_i'] == '1') & (shufflev_trace_df['ready_i'] == '1') & (shufflev_trace_df['valid_o'] == '1') & (shufflev_trace_df['branch_i'] == '0')]

In [ ]:
shufflev_trace_df_filter = shufflev_trace_df_filter.sort_values(by=['fetch_id_idex[31:0]'], key=lambda col: [int(c[1:], base=2) for c in col])

In [ ]:
for t1, t2 in zip(ibex_trace_df_filter.iterrows(), shufflev_trace_df_filter.iterrows()):
    # print (t1[0], hex(int(t1[1]['addr_o[31:0]'][1:], base=2)), hex(int(t1[1]['rdata_o[31:0]'][1:], base=2)), 
    #        t2[0], hex(int(t2[1]['addr_o[31:0]'][1:], base=2)), hex(int(t2[1]['rdata_o[31:0]'][1:], base=2)))
    if (t1[1]['addr_o[31:0]'] != t2[1]['addr_o[31:0]']) or (t1[1]['instr_decompressed[31:0]'] != t2[1]['rdata_o[31:0]']):
        print ('Trace mismatch at timestamp: ', t1[0], t2[0])
        break